# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2022 12:00AM,237774,10,0085694610,ISDIN Corporation,Released
1,May 23 2022 12:00AM,237774,10,0085694611,ISDIN Corporation,Released
2,May 23 2022 12:00AM,237774,10,0085694615,ISDIN Corporation,Released
3,May 23 2022 12:00AM,237774,10,0085694619,ISDIN Corporation,Released
4,May 23 2022 12:00AM,237774,10,0085694614,ISDIN Corporation,Released
5,May 23 2022 12:00AM,237774,10,0085694617,ISDIN Corporation,Released
6,May 23 2022 12:00AM,237774,10,0085694618,ISDIN Corporation,Released
7,May 23 2022 12:00AM,237774,10,0085694624,ISDIN Corporation,Released
8,May 23 2022 12:00AM,237774,10,0085694621,ISDIN Corporation,Released
9,May 23 2022 12:00AM,237774,10,0085694616,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,237773,Completed,1
17,237773,Released,1
18,237774,Completed,1
19,237774,Released,33
20,237784,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237770,NaN,NaN,4.0
237771,NaN,NaN,1.0
237773,1.0,NaN,1.0
237774,1.0,NaN,33.0
237784,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237693,0.0,0.0,1.0
237694,0.0,0.0,1.0
237724,0.0,0.0,1.0
237739,0.0,0.0,9.0
237742,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237693,0,0,1
237694,0,0,1
237724,0,0,1
237739,0,0,9
237742,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237693,0,0,1
1,237694,0,0,1
2,237724,0,0,1
3,237739,0,0,9
4,237742,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237693,,,1
1,237694,,,1
2,237724,,,1
3,237739,,,9
4,237742,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2022 12:00AM,237774,10,ISDIN Corporation
34,May 20 2022 4:15PM,237784,16,"SHL Pharma, LLC"
35,May 20 2022 3:28PM,237773,10,"Methapharm, Inc."
37,May 20 2022 3:19PM,237771,18,"ClearSpec, LLC"
38,May 20 2022 2:45PM,237770,18,Hush Hush
42,May 20 2022 12:57PM,237758,10,ISDIN Corporation
54,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC
55,May 20 2022 12:39PM,237752,18,Hush Hush
56,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC"
57,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2022 12:00AM,237774,10,ISDIN Corporation,1,,33
1,May 20 2022 4:15PM,237784,16,"SHL Pharma, LLC",,,1
2,May 20 2022 3:28PM,237773,10,"Methapharm, Inc.",1,,1
3,May 20 2022 3:19PM,237771,18,"ClearSpec, LLC",,,1
4,May 20 2022 2:45PM,237770,18,Hush Hush,,,4
5,May 20 2022 12:57PM,237758,10,ISDIN Corporation,,11,1
6,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC,,,1
7,May 20 2022 12:39PM,237752,18,Hush Hush,,,1
8,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC",,,1
9,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc.",,,9


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 12:00AM,237774,10,ISDIN Corporation,33,,1
1,May 20 2022 4:15PM,237784,16,"SHL Pharma, LLC",1,,
2,May 20 2022 3:28PM,237773,10,"Methapharm, Inc.",1,,1
3,May 20 2022 3:19PM,237771,18,"ClearSpec, LLC",1,,
4,May 20 2022 2:45PM,237770,18,Hush Hush,4,,
5,May 20 2022 12:57PM,237758,10,ISDIN Corporation,1,11,
6,May 20 2022 12:45PM,237755,20,Reliable 1 Laboratories LLC,1,,
7,May 20 2022 12:39PM,237752,18,Hush Hush,1,,
8,May 20 2022 12:37PM,237754,18,"ClearSpec, LLC",1,,
9,May 20 2022 12:35PM,237739,20,"ACI Healthcare USA, Inc.",9,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 12:00AM,237774,10,ISDIN Corporation,33,,1
1,May 20 2022 4:15PM,237784,16,"SHL Pharma, LLC",1,,
2,May 20 2022 3:28PM,237773,10,"Methapharm, Inc.",1,,1
3,May 20 2022 3:19PM,237771,18,"ClearSpec, LLC",1,,
4,May 20 2022 2:45PM,237770,18,Hush Hush,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 12:00AM,237774,10,ISDIN Corporation,33,NaN,1.0
1,May 20 2022 4:15PM,237784,16,"SHL Pharma, LLC",1,NaN,NaN
2,May 20 2022 3:28PM,237773,10,"Methapharm, Inc.",1,NaN,1.0
3,May 20 2022 3:19PM,237771,18,"ClearSpec, LLC",1,NaN,NaN
4,May 20 2022 2:45PM,237770,18,Hush Hush,4,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2022 12:00AM,237774,10,ISDIN Corporation,33,0.0,1.0
1,May 20 2022 4:15PM,237784,16,"SHL Pharma, LLC",1,0.0,0.0
2,May 20 2022 3:28PM,237773,10,"Methapharm, Inc.",1,0.0,1.0
3,May 20 2022 3:19PM,237771,18,"ClearSpec, LLC",1,0.0,0.0
4,May 20 2022 2:45PM,237770,18,Hush Hush,4,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1188692,37,11.0,2.0
16,475526,2,0.0,0.0
18,1902044,11,0.0,0.0
20,475494,10,0.0,0.0
21,237724,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1188692,37,11.0,2.0
1,16,475526,2,0.0,0.0
2,18,1902044,11,0.0,0.0
3,20,475494,10,0.0,0.0
4,21,237724,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,37,11.0,2.0
1,16,2,0.0,0.0
2,18,11,0.0,0.0
3,20,10,0.0,0.0
4,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,16,Released,2.0
2,18,Released,11.0
3,20,Released,10.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,20,21
Status,,,,,
Completed,2.0,0.0,0.0,0.0,0.0
Executing,11.0,0.0,0.0,0.0,0.0
Released,37.0,2.0,11.0,10.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,11.0,0.0,0.0,0.0,0.0
2,Released,37.0,2.0,11.0,10.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0
1,Executing,11.0,0.0,0.0,0.0,0.0
2,Released,37.0,2.0,11.0,10.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()